In [1]:
from collections import defaultdict
from pymongo import MongoClient
from tqdm import tqdm
from colabfit.tools.database import MongoDatabase

# client = MongoDatabase(database_name="mp", uri="mongodb://localhost:27017/")
client = MongoClient("mongodb://localhost:27017/")
db = client["mp"]
from pprint import pprint


In [30]:
def get_soft_meth(
    batch_size,
    skip,
    last,
    type,
):
    piped = db.property_instances.aggregate(
        [
            {"$match": {"type": type, "hash": {"$gt": last}}},
            {"$sort": {"hash": 1}},
            # {"$match": {"_id":{"$gt": last}}},
            {"$skip": skip},
            {"$limit": batch_size},
            {"$unwind": "$relationships.metadata"},
            {
                "$lookup": {
                    "from": "metadata",
                    "localField": "relationships.metadata",
                    "foreignField": "colabfit-id",
                    "as": "do_data",
                }
            },
            {
                "$lookup": {
                    "from": "data_objects",
                    "localField": "do_data.colabfit-id",
                    "foreignField": "colabfit-id",
                    "as": "data_object",
                }
            },
            {
                "$match": {
                    "data_object.relationships.datasets": {"$ne": ["DS_ifdjgm9le1fd_0"]}
                }
            },
            {
                "$group": {
                    "_id": "$colabfit-id",
                    "hash": {"$push": "$hash"},
                    "md_ids": {"$push": "$relationships.metadata"},
                    "do_ids": {"$push": "$relationships.data_objects"},
                    "method": {"$push": "$do_data.method.source-value"},
                    "software": {"$push": "$do_data.software.source-value"},
                }
            },
            {
                "$project": {
                    "_id": 0,
                    "hash": "$hash",
                    "md_ids": {"$size": "$md_ids"},
                    "do_ids": {"$size": "$do_ids"},
                    "method": "$method",
                    "software": "$software",
                }
            },
        ]
    )
    return piped

In [26]:
def update_ms(ms_data, soft_dict, meth_dict):
    last_hash = None
    for data in ms_data:
        last_hash = data.get("hash")
        if last_hash is None:
            return last_hash
        meth = data["method"][0]
        soft = data["software"][0]
        do_len = data["do_ids"]
        md_len = data["md_ids"]
        # If the number of md-ids == num of do-ids
        if do_len == md_len:
            if len(meth) == 0:
                meth_dict["None"] += do_len
            else:
                if len(meth) == 1:
                    meth_dict[meth[0]] += do_len
                elif len(meth) == do_len:
                    for m in meth:
                        meth_dict[m] += 1
                else:
                    meth_dict["unequal_meth_domd"] += do_len
            if len(soft) == 0:
                soft_dict["None"] += do_len
            else:
                if len(soft) == 1:
                    soft_dict[soft[0]] += do_len
                elif len(soft) == do_len:
                    for s in soft:
                        soft_dict[s] += 1
                else:
                    soft_dict["unequal_soft_domd"] += do_len
        else:
            soft_dict["unequal_do_md"] += 1
            meth_dict["unequal_do_md"] += 1

    return last_hash


In [4]:
methods = defaultdict(int)
software = defaultdict(int)

In [28]:
n_pis = db.property_instances.estimated_document_count()
b_size = 10000
n_batches = n_pis // b_size
if n_batches < 1:
    n_batches = 1
remain = n_pis % b_size

In [33]:
last = "0"

for batch in range(n_batches):
    skip = batch * b_size
    data = get_soft_meth(b_size, skip, last, "free-energy")
    last = update_ms(data, software, methods)
    print(last)
    if last is None:
        break

['54730572218636845']
['54730572218636845']
None


In [ ]:
methods

In [ ]:
pe = []
# for batch in range(n_batches):
#     skip = batch * b_size
# print(skip)
data = db.property_instances.aggregate(
    [
        {"$match": {"type": "potential-energy"}},
        # {"$limit": 1},
        {"$count": "totalDocuments"},
    ]
)
# update_ms(data, software, methods)
print(list(data))

In [ ]:
with open("software_no_oc.txt", "a") as f:
    f.write(str(software))
with open("methods_no_oc.txt", "a") as f:
    f.write(str(methods))

In [ ]:
def main():
    methods = defaultdict(int)
    software = defaultdict(int)
    n_pis = db.property_instances.estimated_document_count()
    b_size = 1000000
    n_batches = n_pis // b_size
    remain = n_pis % b_size
    for batch in range(n_batches):
        skip = batch * b_size
        print(skip)
        data = get_soft_meth(b_size, skip, typ)
        update_ms(data, software, methods)

In [ ]:
with open("pub_dois_2023-6-8.txt", "r") as f:
    dois = f.readlines()
